# Predicting Flu Vaccine Compliance

## Overview

The CDC wants to have a model that can predict who does and who does not get the seasonal flu vaccine. 

They have provided the National 2009 H1N1 Flu Survey Data for the purposes of building a model that can predict who will and won't get the flu vaccine. They hope to use the best features from this model in order to later build other predictive models that can predict compliance with COVID-19 vaccines, as currently the boosters are routine, like the seasonal flu vaccines. 

### Buisness Problem

Creating a model in order to pull out the features that best predict who will get the seasonal flu vaccine. 

### Dataset Size 

The initial dataset was 26,707 rows with 36 columns. After data cleaning there were 27 columns. 

### Limitations of the Dataset

The data was collected via telephone surveys, a commonly used polling method which is not representative or random (as people choose to respond or not when they are called). Additionally, today new models may need to take into account the anti-vaccine movement (article [here](https://pubmed.ncbi.nlm.nih.gov/16039769/)) which was not as prevalent when the data was collected in 2009 - this may affect the accuracy of the models built if applied to more recent datasets. 

### Dataset Size 

The initial dataset was 26,707 rows with 36 columns. After data cleaning there were 27 relevant columns. 

### Limitations Of The Dataset

The data was collected via telephone surveys, a commonly used polling method which is not representative or random (as people choose to respond or not when they are called). Additionally, today new models may need to take into account the anti-vaccine movement (article [here](https://pubmed.ncbi.nlm.nih.gov/16039769/)) which was not as prevalent when the data was collected in 2009 - this may affect the accuracy of the models built now on more current datasets. 


### Why We Used This Dataset

Despite the above limitations, the dataset does contain a large number of responses on and takes into account a large number of features relevant to the seasonal flu vaccine compliance, and is a relatively recent dataset. For all these reasons, we decided to use this dataset to create our predictive model. 


## Looking at the Data

### Imports

In [1]:
# imports
import numpy as np
import pandas as pd
import statistics
import scipy.sparse
#import warnings
#warnings.filterwarnings('ignore')

from scipy.stats import chi2_contingency

from sklearn.preprocessing import FunctionTransformer, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer


### Data: Initial Look

In [2]:
features = pd.read_csv("Data/training_set_features.csv")
labels = pd.read_csv("Data/training_set_labels.csv")

In [3]:
#checking if the features and lable dataframes match up 
np.testing.assert_array_equal(features.index.values, labels.index.values)

In [4]:
#lets look at the features data
features.head()

,respondent_id,h1n1_concern,h1n1_knowledge,behavioral_antiviral_meds,behavioral_avoidance,behavioral_face_mask,behavioral_wash_hands,behavioral_large_gatherings,behavioral_outside_home,behavioral_touch_face,...,income_poverty,marital_status,rent_or_own,employment_status,hhs_geo_region,census_msa,household_adults,household_children,employment_industry,employment_occupation
0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Own,Not in Labor Force,oxchjgsf,Non-MSA,0.0,0.0,NaN,NaN
1,1,3.0,2.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,...,Below Poverty,Not Married,Rent,Employed,bhuqouqj,"MSA, Not Principle City",0.0,0.0,pxcmvdjn,xgwztkwe
2,2,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,"<= $75,000, Above Poverty",Not Married,Own,Employed,qufhixun,"MSA, Not Principle City",2.0,0.0,rucpziij,xtkaffoo
3,3,1.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,...,Below Poverty,Not Married,Rent,Not in Labor Force,lrircsnp,"MSA, Principle City",0.0,0.0,NaN,NaN
4,4,2.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,1.0,...,"<= $75,000, Above Poverty",Married,Own,Employed,qufhixun,"MSA, Not Principle City",1.0,0.0,wxleyezf,emcorrxb


In [5]:
# Now lets look into the labels data
labels.head()

,respondent_id,h1n1_vaccine,seasonal_vaccine
0,0,0,0
1,1,0,1
2,2,0,0
3,3,0,1
4,4,0,0


#### Description of Features

In [6]:
# Looking into the `features` DataFrame
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26707 entries, 0 to 26706
Data columns (total 36 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   respondent_id                26707 non-null  int64  
 1   h1n1_concern                 26615 non-null  float64
 2   h1n1_knowledge               26591 non-null  float64
 3   behavioral_antiviral_meds    26636 non-null  float64
 4   behavioral_avoidance         26499 non-null  float64
 5   behavioral_face_mask         26688 non-null  float64
 6   behavioral_wash_hands        26665 non-null  float64
 7   behavioral_large_gatherings  26620 non-null  float64
 8   behavioral_outside_home      26625 non-null  float64
 9   behavioral_touch_face        26579 non-null  float64
 10  doctor_recc_h1n1             24547 non-null  float64
 11  doctor_recc_seasonal         24547 non-null  float64
 12  chronic_med_condition        25736 non-null  float64
 13  child_under_6_mo

Descriptions of the features (taken from [here](https://www.drivendata.org/competitions/66/flu-shot-learning/page/211/)): 

For all binary variables: 0 = No; 1 = Yes.

- h1n1_concern - Level of concern about the H1N1 flu. 0 = Not at all concerned; 1 = Not very concerned; 2 = Somewhat concerned; 3 = Very concerned.
- h1n1_knowledge - Level of knowledge about H1N1 flu. 0 = No knowledge; 1 = A little knowledge; 2 = A lot of knowledge.
- behavioral_antiviral_meds - Has taken antiviral medications. (binary)
- behavioral_avoidance - Has avoided close contact with others with flu-like symptoms. (binary)
- behavioral_face_mask - Has bought a face mask. (binary)
- behavioral_wash_hands - Has frequently washed hands or used hand sanitizer. (binary)
- behavioral_large_gatherings - Has reduced time at large gatherings. (binary)
- behavioral_outside_home - Has reduced contact with people outside of own household. (binary)
- behavioral_touch_face - Has avoided touching eyes, nose, or mouth. (binary)
- doctor_recc_h1n1 - H1N1 flu vaccine was recommended by doctor. (binary)
- doctor_recc_seasonal - Seasonal flu vaccine was recommended by doctor. (binary)
- chronic_med_condition - Has any of the following chronic medical conditions: asthma or an other lung condition, diabetes, a heart condition, a kidney condition, sickle cell anemia or other anemia, a neurological or neuromuscular condition, a liver condition, or a weakened immune system caused by a chronic illness or by medicines taken for a chronic illness. (binary)
- child_under_6_months - Has regular close contact with a child under the age of six months. (binary)
- health_worker - Is a healthcare worker. (binary)
- health_insurance - Has health insurance. (binary)
- opinion_h1n1_vacc_effective - Respondent's opinion about H1N1 vaccine effectiveness. 1 = Not at all effective; 2 = Not very effective; 3 = Don't know; 4 = Somewhat effective; 5 = Very effective.
- opinion_h1n1_risk - Respondent's opinion about risk of getting sick with H1N1 flu without vaccine.1 = Very Low; 2 = Somewhat low; 3 = Don't know; 4 = Somewhat high; 5 = Very high.
- opinion_h1n1_sick_from_vacc - Respondent's worry of getting sick from taking H1N1 vaccine.1 = Not at all worried; 2 = Not very worried; 3 = Don't know; 4 = Somewhat worried; 5 = Very worried.
- opinion_seas_vacc_effective - Respondent's opinion about seasonal flu vaccine effectiveness.1 = Not at all effective; 2 = Not very effective; 3 = Don't know; 4 = Somewhat effective; 5 = Very effective.
- opinion_seas_risk - Respondent's opinion about risk of getting sick with seasonal flu without vaccine.1 = Very Low; 2 = Somewhat low; 3 = Don't know; 4 = Somewhat high; 5 = Very high.
- opinion_seas_sick_from_vacc - Respondent's worry of getting sick from taking seasonal flu vaccine.1 = Not at all worried; 2 = Not very worried; 3 = Don't know; 4 = Somewhat worried; 5 = Very worried.
- age_group - Age group of respondent.
- education - Self-reported education level.
- race - Race of respondent.
- sex - Sex of respondent.
- income_poverty - Household annual income of respondent with respect to 2008 Census poverty thresholds.
- marital_status - Marital status of respondent.
- rent_or_own - Housing situation of respondent.
- employment_status - Employment status of respondent.
- hhs_geo_region - Respondent's residence using a 10-region geographic classification defined by the U.S. Dept. of Health and Human Services. Values are represented as short random character strings.
- census_msa - Respondent's residence within metropolitan statistical areas (MSA) as defined by the U.S. Census.
- household_adults - Number of other adults in household, top-coded to 3.
- household_children - Number of children in household, top-coded to 3.
- employment_industry - Type of industry respondent is employed in. Values are represented as short random character strings.
- employment_occupation - Type of occupation of respondent. Values are represented as short random character strings.

### Data: Initial Cleaning

Let's remove all of the columns related to only the H1N1 vaccine: h1n1_concern, h1n1_knowledge, doctor_recc_h1n1, opinion_h1n1_vacc_effective, opinion_h1n1_risk, opinion_h1n1_sick_from_vacc

Additionally, there are some columns where the information has been scrambled (presumably to protect the respondents personal information) so lets remove those as well as we can't extract any useful information without knowing what they are coded for: hhs_geo_region, employment_industry, employment_occupation

Finally, let's remove 'h1n1_vaccine' and 'respondent_id' from the labels DataFrame, and then turn the labels dataframe into an array so we can use it when we build our models later on. 

In [7]:
features.drop(['h1n1_concern', 'h1n1_knowledge', 'doctor_recc_h1n1',
               'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk', 
               'opinion_h1n1_sick_from_vacc', 'hhs_geo_region', 
               'employment_industry', 'employment_occupation'], axis = 1, inplace = True)

labels.drop(['h1n1_vaccine', 'respondent_id'], axis = 1, inplace= True)
labels = np.ravel(labels, order = 'C')

In [8]:
# train_test_split()
X_train, X_test, y_train, y_test = train_test_split(features, labels, random_state = 42)

## Creating Preprocessing Pipeline

In order to create a pipeline for preprocessing, let's first figure out how we will handle our missing values.

In [9]:
features.isnull().sum()

respondent_id                      0
behavioral_antiviral_meds         71
behavioral_avoidance             208
behavioral_face_mask              19
behavioral_wash_hands             42
behavioral_large_gatherings       87
behavioral_outside_home           82
behavioral_touch_face            128
doctor_recc_seasonal            2160
chronic_med_condition            971
child_under_6_months             820
health_worker                    804
health_insurance               12274
opinion_seas_vacc_effective      462
opinion_seas_risk                514
opinion_seas_sick_from_vacc      537
age_group                          0
education                       1407
race                               0
sex                                0
income_poverty                  4423
marital_status                  1408
rent_or_own                     2042
employment_status               1463
census_msa                         0
household_adults                 249
household_children               249
d

The largest amount of missing data is in `health_insurance` - let's look into the signifigance of that variable to see if we can drop it. 

In [10]:
cross_tab = pd.crosstab(features['health_insurance'], labels, margins = True)
print(cross_tab)
chi2, p, dof, expected = chi2_contingency(cross_tab)
print(chi2, p)

col_0                0     1    All
health_insurance                   
0.0               1338   398   1736
1.0               5866  6831  12697
All               7204  7229  14433
582.2867466798792 1.0559310374176207e-124


We see above a statistically significant p-value and a very high chi2 value. As such, I've decided to keep this feature. 

I've chosen to change all NaN's to 0, as in 2009 when the data was collected the Affordable Care Act had not been passed, so people were not required to have health insurance. I think that keeping the NaN's as 0's is better reflective of this time. 

As for the rest of the missing variables:

 - For the rest of the **binary data** let's also replace all of the missing data with 0's. 
 
 - For the opinion questions and family features (`household_adults` and `household_children`) which are numeric **ordinal and interval data** let's replace all the missing data with the median. Additionally, we will use `MinMaxScaler` to scale this data. Most of our data is binary, and `MinMaxScaler` will keep the scaled data in the range of 0-1, which is ideal in this case. 
 
 - Finally, for our **categorical data** let's use `OneHotEncoder` to create dummy categories
 
 We will put all of this into a pipeline, so we can test out different models!  


In [11]:
#create functions for preprocessing

# function to replace NaN's in the ordinal and interval data 
def replace_NAN_median(X_df):
    opinions = ['opinion_seas_vacc_effective', 'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'household_adults',
                'household_children']
    for column in opinions:
        X_df[column].replace(np.nan, X_df[column].median(), inplace = True)
    return X_df

# function to replace NaN's in the catagorical data     
def replace_NAN_mode(X_df):
    miss_cat_features = ['education', 'income_poverty', 'marital_status', 'rent_or_own', 'employment_status']
    for column in miss_cat_features:
        X_df[column].replace(np.nan, statistics.mode(X_df[column]), inplace = True)
    return X_df

        

In [12]:
# Instantiate transformers
NAN_median = FunctionTransformer(replace_NAN_median)
NAN_mode = FunctionTransformer(replace_NAN_mode)
col_transformer = ColumnTransformer(transformers=
    # replace NaN's in the binary data                                
    [("NAN_0", SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 0), 
    ['behavioral_antiviral_meds', 'behavioral_avoidance','behavioral_face_mask' ,
    'behavioral_wash_hands', 'behavioral_large_gatherings', 'behavioral_outside_home',
    'behavioral_touch_face', 'doctor_recc_seasonal', 'chronic_med_condition', 
    'child_under_6_months', 'health_worker', 'health_insurance']),
    
     # MinMaxScaler on our numeric ordinal and interval data
    ("scaler", MinMaxScaler(), ['opinion_seas_vacc_effective', 'opinion_seas_risk',
                                'opinion_seas_sick_from_vacc', 
                                'household_adults', 'household_children']),
     
     # OHE catagorical string data
    ("ohe", OneHotEncoder(sparse = False), ['age_group','education', 'race', 'sex', 
                                'income_poverty', 'marital_status', 'rent_or_own',
                                'employment_status', 'census_msa'])],
     
    remainder="passthrough")



In [13]:
# Preprocessing Pipeline (Yey!)
preprocessing_pipe = Pipeline(steps=[
    ("NAN_median", NAN_median), 
    ("NAN_mode", NAN_mode), 
    ("col_transformer", col_transformer)
    ])

## Modeling

### Create Baseline Model (LogisticRegression())

Because our outcome data is binary, let's try using `LogisticRegression` to model our data.

In [14]:
# Using our pipeline to instantiate the first model
logreg_base_model_pipe = Pipeline(steps=[("preprocessing_pipe", preprocessing_pipe),
                                    ("log_reg", LogisticRegression(random_state = 42))])
    

In [15]:
# fitting the model to our training data
logreg_base_model_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing_pipe',
                 Pipeline(steps=[('NAN_median',
                                  FunctionTransformer(func=<function replace_NAN_median at 0x000001A89012E550>)),
                                 ('NAN_mode',
                                  FunctionTransformer(func=<function replace_NAN_mode at 0x000001A89012E670>)),
                                 ('col_transformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('NAN_0',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy=...
                                                                  ('scaler',
                                                                   MinMaxScaler(),
                                                                   ['opinion_seas_vacc

In [16]:
# getting the mean accuracy of the model
logreg_base_model_pipe.score(X_train, y_train)


0.5310534198701947

### Optimized LogisticRegression() Model
In our baseline `LogisticRegression` model, we got .53% accuracy - hardly better than a coin flip. 

Lets see if changing the parameters `solver`, `penalty`(if we use L1 (Lasso) or L2 (Ridge), and `C` (how strong the regularization strength is with smaller values being *stronger* regularization) will improve our accuracy. 

We can check all these things at once using `GridSearchCV`

In [17]:
param_grid = {
    'log_reg__solver': ['liblinear'],
    'log_reg__penalty': ['l1', 'l2'], 
    'log_reg__C': [0.001,0.01,0.1,1,10,100,1000]   
}

gs = GridSearchCV(estimator=logreg_base_model_pipe,
                  param_grid=param_grid,
                  cv=5)


In [18]:
#logreg_model_pipe.get_params().keys()


In [19]:
gs.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing_pipe',
                                        Pipeline(steps=[('NAN_median',
                                                         FunctionTransformer(func=<function replace_NAN_median at 0x000001A89012E550>)),
                                                        ('NAN_mode',
                                                         FunctionTransformer(func=<function replace_NAN_mode at 0x000001A89012E670>)),
                                                        ('col_transformer',
                                                         ColumnTransformer(remainder='passthrough',
                                                                           transformers=[('NAN_0',
                                                                                          SimpleIm...
                                                                                           'household_children']),
               

In [20]:
# Finding the parameters with the best results 
gs.best_params_

{'log_reg__C': 10, 'log_reg__penalty': 'l1', 'log_reg__solver': 'liblinear'}

In [21]:
# creating a new model with the optimized parameters
logreg_optimized_pipe =  Pipeline(steps=[("preprocessing_pipe", preprocessing_pipe),
                                    ("log_reg", LogisticRegression(solver = 'liblinear', random_state = 42, C = 10, penalty= 'l1'))])
    

In [22]:
logreg_optimized_pipe.fit(X_train, y_train)

Pipeline(steps=[('preprocessing_pipe',
                 Pipeline(steps=[('NAN_median',
                                  FunctionTransformer(func=<function replace_NAN_median at 0x000001A89012E550>)),
                                 ('NAN_mode',
                                  FunctionTransformer(func=<function replace_NAN_mode at 0x000001A89012E670>)),
                                 ('col_transformer',
                                  ColumnTransformer(remainder='passthrough',
                                                    transformers=[('NAN_0',
                                                                   SimpleImputer(fill_value=0,
                                                                                 strategy=...
                                                                   ['opinion_seas_vacc_effective',
                                                                    'opinion_seas_risk',
                                                          

In [23]:
logreg_optimized_pipe.score(X_train, y_train)

0.7741387918122816

In [24]:
# code inspiration taken from: 
# https://stackoverflow.com/questions/38787612/how-to-extract-feature-importances-from-an-sklearn-pipeline
logreg_optimized_pipe.steps[1][1].coef_

array([[ 7.76108305e-02, -4.71093522e-02,  1.71787476e-03,
         1.45673325e-01, -2.53400628e-03, -4.56066968e-02,
         2.49698996e-01,  1.29665130e+00,  1.79070203e-01,
         3.60191279e-02,  8.09461790e-01,  4.31263467e-01,
         2.30594856e+00,  2.15343491e+00, -8.97631090e-01,
        -2.00116903e-01, -1.51416275e-01, -8.72202563e-01,
        -6.80397946e-01, -4.89292814e-01, -2.17216603e-01,
         5.78978247e-01, -3.09803799e-01, -4.80350862e-01,
        -5.86653093e-02, -2.45671390e-01, -6.64758153e-01,
        -4.72492187e-01, -2.91403381e-01, -3.27679681e-01,
        -4.22607616e-01, -4.09108570e-01, -3.56995968e-01,
        -2.30700681e-01, -4.96987046e-01, -3.27058555e-01,
        -4.68740678e-01, -1.56218641e-01, -3.70495135e-01,
        -6.08972569e-01, -5.45126503e-01, -7.91206615e-01,
        -2.51446403e-01, -2.58559662e-01, -3.85708419e-01,
        -1.66229768e-06]])

In [25]:
# https://stackoverflow.com/questions/54646709/sklearn-pipeline-get-feature-names-after-onehotencode-in-columntransformer
logreg_optimized_pipe.named_steps["log_reg"].feature_names_in_
#ohe_names = list(logreg_optimized_pipe.named_steps["preprocessing_pipe"][2].transformers_[2][1].get_feature_names_out())
#ohe_names


AttributeError: 'LogisticRegression' object has no attribute 'feature_names_in_'

In [26]:
coefficients = logreg_optimized_pipe.steps[1][1].coef_[0]
# summarize feature importance
for i,v in enumerate(coefficients):
    print('Feature: %0d, Score: %.5f' % (i,v))

Feature: 0, Score: 0.07761
Feature: 1, Score: -0.04711
Feature: 2, Score: 0.00172
Feature: 3, Score: 0.14567
Feature: 4, Score: -0.00253
Feature: 5, Score: -0.04561
Feature: 6, Score: 0.24970
Feature: 7, Score: 1.29665
Feature: 8, Score: 0.17907
Feature: 9, Score: 0.03602
Feature: 10, Score: 0.80946
Feature: 11, Score: 0.43126
Feature: 12, Score: 2.30595
Feature: 13, Score: 2.15343
Feature: 14, Score: -0.89763
Feature: 15, Score: -0.20012
Feature: 16, Score: -0.15142
Feature: 17, Score: -0.87220
Feature: 18, Score: -0.68040
Feature: 19, Score: -0.48929
Feature: 20, Score: -0.21722
Feature: 21, Score: 0.57898
Feature: 22, Score: -0.30980
Feature: 23, Score: -0.48035
Feature: 24, Score: -0.05867
Feature: 25, Score: -0.24567
Feature: 26, Score: -0.66476
Feature: 27, Score: -0.47249
Feature: 28, Score: -0.29140
Feature: 29, Score: -0.32768
Feature: 30, Score: -0.42261
Feature: 31, Score: -0.40911
Feature: 32, Score: -0.35700
Feature: 33, Score: -0.23070
Feature: 34, Score: -0.49699
Feature

In [ ]:
coefficients_df = pd.concat([pd.DataFrame(X_train.columns), pd.DataFrame(np.transpose(coefficients))], axis = 1)
coefficients_df.columns = ["feature_names", "coefficients"]
coefficients_df

In [ ]:
# https://stackoverflow.com/questions/51328373/pandas-replace-nan-values-with-a-list-for-specific-columns
coefficients_df.loc[coefficients_df["feature_names"].isnull(), "feature_names"] = ohe_names

In [ ]:
coefficients_df['feature_names'].isnull().sum()